In [1]:
from disqco.circuits.cp_fraction import cp_fraction, cz_fraction
from disqco.graphs.QC_hypergraph import QuantumCircuitHyperGraph
from disqco.graphs.hypergraph_methods import calculate_full_cost_hetero
from qiskit import transpile
from copy import deepcopy
from disqco.graphs.quantum_network import *
from copy import deepcopy
import numpy as np
from disqco.graphs.quantum_network import QuantumNetwork
import os
import json

num_qubits_list = [128,256,512]
network_types = ['linear', 'grid']
num_partitions_and_coarsening_factors = [(4, 2), (8, 2), (16, 2), (32, 2), (64, 2), (8, 4), (16, 4), (32, 4), (64, 4), (16, 8), (32, 8), (64, 8)]

###############################################################################
# Set up JSON file for storing *all* iteration results (detailed data)
###############################################################################
detailed_filename = "benchmark_results_COARENING_POST_PROCESSED_nolimit2.json"

if os.path.exists(detailed_filename):
    with open(detailed_filename, "r") as f:
        detailed_results = json.load(f)
else:
    detailed_results = []

In [ ]:
from disqco.parti.FM.net_coarsened_partitioning import run_full_net_coarsened_FM
from disqco.parti.FM.FM_methods import set_initial_partition_assignment
from disqco.graphs.quantum_network import linear_coupling, grid_coupling
import time

iterations = 5
for iteration in range(iterations):
    for num_qubits in num_qubits_list:
        for network_type in network_types:
            for num_partitions_and_coarsening_factor in num_partitions_and_coarsening_factors:
                level_limit = 100  # Set the level limit for coarsening
                num_partitions, coarsening_factor = num_partitions_and_coarsening_factor
                print(f"num_qubits: {num_qubits}, network_type: {network_type}, num_partitions: {num_partitions}, coarsening_factor: {coarsening_factor}")
                # Check first whether the json file already contains results for this configuration
                
                seed = np.random.randint(0, 10000)
                
                circuit = cp_fraction(num_qubits=num_qubits,
                                        depth=num_qubits,
                                        fraction=0.5,
                                        seed=seed)
                
                circuit = transpile(circuit, basis_gates=['cp', 'u'])

                qpu_sizes = [int(np.ceil(num_qubits / num_partitions)) + 1 for i in range(num_partitions)]

                if network_type == 'linear':
                    coupling = linear_coupling(num_partitions)
                elif network_type == 'grid':
                    coupling = grid_coupling(num_partitions)

                initial_qpu_sizes = {i: qpu_sizes[i] for i in range(num_partitions)}
                initial_network = QuantumNetwork(qpu_sizes, coupling)

                network = deepcopy(initial_network)
                initial_graph = QuantumCircuitHyperGraph(circuit, group_gates=True, anti_diag=True)
                depth = initial_graph.depth
                graph_no_grouping = QuantumCircuitHyperGraph(circuit, group_gates=False, anti_diag=True)

                graph = deepcopy(initial_graph)
                assignment = set_initial_partition_assignment(graph, network)

                initial_cost_no_grouping = calculate_full_cost_hetero(hypergraph=graph_no_grouping, assignment=assignment, num_partitions=num_partitions, costs = {}, network=network)

                print("Initial cost with no grouping: ", initial_cost_no_grouping)

                initial_cost = calculate_full_cost_hetero(hypergraph=graph, assignment=assignment, num_partitions=num_partitions, costs = {}, network=network)

                print("Cost with grouping: ", initial_cost)


                start_time = time.time()
                results  = run_full_net_coarsened_FM(
                                                    hypergraph=graph,
                                                    num_qubits=num_qubits,
                                                    network=network,
                                                    coarsening_factor=coarsening_factor,
                                                    passes_per_level=10,
                                                    use_multiprocessing=True,
                                                    ML_internal_level_limit=100)
                
                final_assignment = results['best_assignment']
                final_cost = results['best_cost']

                end_time = time.time()
                print("Final Cost: ", final_cost)
                print("Time taken: ", end_time - start_time)
                results_entry = {
                    "num_qubits": num_qubits,
                    "network_type": network_type,
                    "num_partitions": num_partitions,
                    "coarsening_factor": coarsening_factor,
                    "level_limit": level_limit,
                    "initial_cost_no_grouping": initial_cost_no_grouping,
                    "initial_cost": initial_cost,
                    "final_cost": final_cost,
                    "iteration": iteration,
                    "time_taken": end_time - start_time,
                    "seed": seed
                }
                detailed_results.append(results_entry)

                detailed_results.append(results_entry)
                if level_limit > 1:
                    level_limit -= 1
                        
                        
                with open(detailed_filename, "w") as f:
                    json.dump(detailed_results, f, indent=2)
